In [1]:
print("hello world")

hello world


In [2]:
!pip install protobuf==3.20.3
!pip install --upgrade tokenizers
!pip install crewai crewai_tools langchain_community python-dotenv

                                              0.0/162.1 kB ? eta -:--:--
     -------------------------------------- 162.1/162.1 kB 4.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



                                              0.0/2.4 MB ? eta -:--:--
     ---                                      0.2/2.4 MB 6.9 MB/s eta 0:00:01
     ---------                                0.6/2.4 MB 7.1 MB/s eta 0:00:01
     -----------------                        1.0/2.4 MB 8.2 MB/s eta 0:00:01
     ---------------------------              1.6/2.4 MB 9.3 MB/s eta 0:00:01
     --------------------------------------   2.3/2.4 MB 10.5 MB/s eta 0:00:01
     ---------------------------------------- 2.4/2.4 MB 10.2 MB/s eta 0:00:00
                                              0.0/450.5 kB ? eta -:--:--
     ------------------------------------- 450.5/450.5 kB 27.5 MB/s eta 0:00:00
  Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
                                              0.0/183.9 kB ? eta -:--:--
     ------------------------------------- 183.9/183.9 kB 10.9 MB/s eta 0:00:00
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl (161 kB)
  Using cached requests-2.32.3-py


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from google.colab import userdata
import os
import litellm

In [ ]:
input_dir = "original-pics"  # Replace with your input folder
downscaled_dir = "downscaled-pics"  # Replace with your output folder
selected_dir = "selected-pics"  # Replace with your output folder
example_web_files = "/content/example-markdown"

In [ ]:
# task_hello_world = Task(
#     description=(
#         """Print a poem on 5 lines, detailing the struggles of debugging code"""
#     ),
#     expected_output="""A poem, 5 lines""",
#     agent=agent_image_extractor
# )

In [ ]:
## START HERE

from PIL import Image
import imghdr

def resize_images(input_folder, output_folder, max_dimension=1080):
    """
    Resizes images in the input folder, ensuring the largest dimension is max_dimension,
    maintaining aspect ratio, and saves them to the output folder.

    Args:
      input_folder: Path to the input folder containing images.
      output_folder: Path to the output folder where resized images will be saved.
      max_dimension: The maximum size for the largest dimension (width or height).
    """

    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file before checking image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with Image.open(image_path) as img:
                    width, height = img.size

                    # Calculate new dimensions based on the largest dimension
                    if width > height:
                        new_width = max_dimension
                        new_height = int(height * (max_dimension / width))
                    else:
                        new_height = max_dimension
                        new_width = int(width * (max_dimension / height))

                    resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)  # Use LANCZOS for better quality
                    output_path = os.path.join(output_folder, filename)
                    resized_img.save(output_path)
                    print(f"Resized {filename} and saved to {output_folder}")
            except Exception as e:  # Handle potential errors
                print(f"Error processing {filename}: {e}")


resize_images(input_dir, downscaled_dir)

Resized P9021081.jpg and saved to downscaled-pics
Resized P9021062.jpg and saved to downscaled-pics
Resized P3040178.jpg and saved to downscaled-pics
Resized P5030087.jpg and saved to downscaled-pics
Resized P4170159.jpg and saved to downscaled-pics


In [37]:
from PIL import Image
import imghdr
import os

def resize_images(input_folder, output_folder, max_dimension=1080):
    """
    Resizes images in the input folder, ensuring the largest dimension is max_dimension,
    maintaining aspect ratio, and saves them to the output folder, preserving date taken metadata.

    Args:
      input_folder: Path to the input folder containing images.
      output_folder: Path to the output folder where resized images will be saved.
      max_dimension: The maximum size for the largest dimension (width or height).
    """

    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file before checking image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with Image.open(image_path) as img:
                    exif_data = img.getexif()  # Get EXIF data
                    date_taken = exif_data.get(36867) if exif_data else None  # Extract date taken

                    width, height = img.size

                    # Calculate new dimensions based on the largest dimension
                    if width > height:
                        new_width = max_dimension
                        new_height = int(height * (max_dimension / width))
                    else:
                        new_height = max_dimension
                        new_width = int(width * (max_dimension / height))

                    resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)  # Use LANCZOS for better quality

                    # Preserve EXIF data in the resized image
                    if date_taken:
                      resized_img.info['exif'] = img.info['exif']

                    output_path = os.path.join(output_folder, filename)
                    resized_img.save(output_path, exif=resized_img.info.get('exif'))  # Save with EXIF data
                    print(f"Resized {filename} and saved to {output_folder}")
            except Exception as e:  # Handle potential errors
                print(f"Error processing {filename}: {e}")


resize_images(input_dir, downscaled_dir)

Resized P9021081.jpg and saved to downscaled-pics
Resized P9021062.jpg and saved to downscaled-pics
Resized P3040178.jpg and saved to downscaled-pics
Resized P5030087.jpg and saved to downscaled-pics
Resized P4170159.jpg and saved to downscaled-pics
Resized P5270907.jpg and saved to downscaled-pics
Resized P5260235.jpg and saved to downscaled-pics


In [38]:
# prompt: print the date taken metadata from all the images in input_dir

from PIL import Image
import os

def print_image_date_taken(input_dir):
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            filepath = os.path.join(input_dir, filename)
            try:
                img = Image.open(filepath)
                exif_data = img._getexif()
                if exif_data:
                    date_taken = exif_data.get(36867) # 36867 is the EXIF tag for DateTimeOriginal
                    if date_taken:
                        print(f"{filename}: {date_taken}")
                    else:
                        print(f"{filename}: Date taken not found in EXIF data")
                else:
                    print(f"{filename}: No EXIF data found")
            except (IOError, OSError, AttributeError) as e:
                print(f"Error processing {filename}: {e}")

print_image_date_taken(input_dir)
print_image_date_taken(downscaled_dir)

P9021081.jpg: 2024:09:02 07:33:19
P9021062.jpg: 2024:09:02 07:31:58
P3040178.jpg: 2022:03:04 16:47:34
P5030087.jpg: 2022:05:03 19:56:51
P4170159.jpg: 2022:04:17 13:40:51
P5270907.jpg: 2022:05:27 15:06:05
P5260235.jpg: 2022:05:26 13:45:13
P9021081.jpg: 2024:09:02 07:33:19
P9021062.jpg: 2024:09:02 07:31:58
P3040178.jpg: 2022:03:04 16:47:34
P5030087.jpg: 2022:05:03 19:56:51
P4170159.jpg: 2022:04:17 13:40:51
P5270907.jpg: 2022:05:27 15:06:05
P5260235.jpg: 2022:05:26 13:45:13


In [ ]:
import os
import base64
import imghdr

def images_to_base64_dict(input_folder):
    """
    Converts images in the input folder to a dictionary with image names as keys
    and their base64 encoded values as values.

    Args:
      input_folder: Path to the input folder containing images.

    Returns:
      A dictionary where keys are image filenames and values are base64 encoded strings.
    """

    image_dict = {}

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file and a valid image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with open(image_path, "rb") as image_file:
                    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
                    image_dict[filename] = encoded_string
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    return image_dict

# Example usage:
downscaled_dir = "downscaled-pics"  # Replace with your input folder
image_dictionary = images_to_base64_dict(downscaled_dir)

# Print the dictionary (optional)
print(image_dictionary)

{'P9021081.jpg': '...', 'P9021062.jpg': '...', 'P3040178.jpg': '...', 'P5030087.jpg': '...', 'P4170159.jpg': '...'}


In [ ]:
agent_critic_background = """You are a highly respected art critic with 20 years of experience in the art market and 30 years
                          as a practicing artist. In your youth, you explored a wide range of artistic mediums,
                          from painting and sculpture to photography and performance art, giving you a deep understanding
                          of the creative process. Your passion for art has led you to travel the world,
                          immersing yourself in diverse cultures and artistic traditions. You possess a profound
                          knowledge of art history, from ancient cave paintings to contemporary installations,
                          and you are constantly seeking new knowledge through lectures, workshops, and exhibitions.
                          Your critical approach is characterized by a keen eye for visual storytelling,
                          an appreciation for the artist's process, and an openness to experimentation.
                          You are a master of articulating your thoughts in a clear and engaging manner,
                          providing insightful critiques that are both objective and constructive."""

agent_critic_task = """Please provide a detailed and objective analysis of the following image,
                        your analysis should include:
                      1. Objective Description:
                        - Provide a neutral and factual description of the image's subject matter, composition,
                            and key visual elements. Avoid subjective interpretations or emotional responses at this stage.
                        - Pay close attention to the use of color, line, form, texture, and space.
                      2. Critical Assessment:
                        - Your assessment will be based on how good you believe others will find the art work. If
                            you yourself like a picture but you think others don't find it striking or interesting
                            the image will get a lower score.
                        - Technical Execution: Analyze the artist's technical skill in handling the chosen medium
                            and their mastery of fundamental artistic principles (e.g., perspective, proportion,
                            light and shadow).
                        - Composition and Design: Evaluate the effectiveness of the composition in guiding the
                            viewer's eye and creating visual interest. Consider the use of balance, rhythm,
                            and focal points.
                        - Color Palette: Discuss the artist's use of color, noting the dominant hues, contrasts,
                            and their impact on the overall mood and atmosphere.
                        - Conceptual Exploration: If applicable, delve into the conceptual underpinnings of the work.
                            Identify any symbolism, metaphors, or narratives that contribute to its meaning.
                        - Originality and Innovation: Comment on the originality of the work and whether it
                            demonstrates a unique approach or expands upon existing artistic conventions.¨
                      3. Overall Impression:
                        - Summarize your overall impression of the artwork, highlighting its strengths and weaknesses.
                        - Offer constructive suggestions for improvement, if applicable.
                        - If an image is not good, boring, or poor, you are not afraid to say so.
                      4. Objective Quality Score:
                        - Assign a score between 1 and 10 (10 being the highest) based solely on your objective
                            assessment of the criteria above.
                        - It is crucial that your score is free from personal bias or preference. Focus on the
                            artwork's technical merits, compositional strength, and conceptual depth.
                        - If an image is not good, boring, or poor, you are not afraid to say so.
                        - Your score will be based on how good you believe others will find the art work. If
                            you yourself like a picture but you think others don't find it striking or interesting
                            the image will get a lower score.

                      Remember: Your role is to provide an objective and insightful critique, without aiming to please
                          your assessment is attachedd to your reputation, so you focus on being objective.
                          If an image is not good, boring, or poor, you are not afraid to say so."""

In [ ]:
import litellm
import json
import os
import time

critic_prompt = agent_critic_background + "\n\n" + agent_critic_task

def analyze_images(image_dictionary):
    """
    Analyzes images in a dictionary, sending them to Gemini for description,
    critical assessment, and quality score.

    Args:
        image_dictionary: A dictionary where keys are image names and values
                           are base64 encoded image strings.

    Returns:
        A dictionary containing analysis results for each image, following
        the defined schema.
    """

    os.environ['GEMINI_API_KEY'] = userdata.get("GOOGLE_API_KEY")

    results = {}

    for image_name, base64_image in image_dictionary.items():
        response = litellm.completion(
            # model="gemini/gemini-1.5-pro",
            model="gemini/gemini-1.5-flash",
            temperature=0.5,
            max_tokens=4096,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": critic_prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                        },
                    ],
                }
            ],
            # api_key=userdata.get("GOOGLE_API_KEY"),  # Replace with your API key source if needed
            response_format={
                "type": "json_object",
                "response_schema": {
                    "type": "object",
                    "properties": {
                        "description": {"type": "string"},
                        "critical_assessment": {"type": "string"},
                        "quality_score": {"type": "integer", "minimum": 1, "maximum": 10},
                    },
                    "required": ["description", "critical_assessment", "quality_score"],
                }
            }
        )

        try:
            analysis = json.loads(response.choices[0].message.content)
            results[image_name] = analysis
        except json.JSONDecodeError:
            print(f"Error decoding JSON response for image: {image_name}")

        time.sleep(2)  # Introduce a 2-second delay between API calls. not best practice but works for now

    return results

# Example usage:
# image_dictionary = {"image1.jpg": base64_image_string1, "image2.png": base64_image_string2, ...}
analysis_results = analyze_images(image_dictionary)
analysis_results_2 = analyze_images(image_dictionary)
analysis_results_3 = analyze_images(image_dictionary)
print(analysis_results)

{'P9021081.jpg': {'critical_assessment': "The photograph presents a classic white convertible Peugeot 404 parked in front of the Panthéon in Paris. The composition uses a low angle, emphasizing the car's size relative to the imposing architecture. The color palette is simple, with the white car contrasting against the warm tones of the building and the blue sky. Technically, the image is well-executed, with sharp focus and good exposure. The composition is effective in creating a sense of scale and grandeur. However, the concept is somewhat straightforward, lacking a deeper narrative or symbolic layer.  While aesthetically pleasing, its originality is limited as similar images of classic cars in front of iconic buildings are common. The overall impact is one of elegant simplicity, but it may not resonate strongly with viewers seeking more conceptual depth or originality. ", 'description': 'The image shows a white Peugeot 404 convertible parked on a cobblestone street in front of the Pa

In [ ]:
for key, value in analysis_results.items():
    # Accessing the 'quality_score' directly for averaging and updating
    quality_score = value.get('quality_score', 0)  # Using get() with default 0 for safety
    avg_quality_score = (quality_score + analysis_results_2[key].get('quality_score', 0) + analysis_results_3[key].get('quality_score', 0)) / 3.0

    # Updating the 'quality_score' in the original dictionary
    analysis_results[key]['quality_score'] = avg_quality_score
    # print(f"{key}    {quality_score}") # Printing the original quality score before update
    print(analysis_results[key]['quality_score'])

8.0
8.333333333333334
4.666666666666667
6.0
5.666666666666667


In [ ]:
# prompt: save dictionary to txt file, with some formatting or line spaces that make it easy to read

import json
import os

def save_dict_to_txt(data, filename):
  """Saves a dictionary to a text file with formatted output.

  Args:
    data: The dictionary to save.
    filename: The name of the file to save to.
  """
  try:
    with open(filename, 'w') as file:
      for key, value in data.items():
        file.write(f"{key}:\n\n")
        if isinstance(value, dict):
          for k, v in value.items():
            file.write(f"  {k}: {v}\n\n")
        elif isinstance(value, list):
            file.write("  [\n\n")
            for item in value:
                file.write(f"    {item},\n\n")
            file.write("  ]\n\n")
        else:
          file.write(f"  {value}\n\n")
        file.write("\n\n")  # Add an extra newline for better readability

  except Exception as e:
      print(f"An error occurred: {e}")

# Example usage (assuming 'analysis_results' is defined as in your original code)
# Replace with your actual dictionary and file name
save_dict_to_txt(analysis_results, "image_analysis_results.txt")

In [ ]:
# prompt: filter the dictionary analysis_results to only keep elements where quality_score is equal to 8 or higher

filtered_results = {key: value for key, value in analysis_results.items() if value.get('quality_score', 0) >= 7.5}
filtered_results

{'P9021081.jpg': {'critical_assessment': "The photograph presents a classic white convertible Peugeot 404 parked in front of the Panthéon in Paris. The composition uses a low angle, emphasizing the car's size relative to the imposing architecture. The color palette is simple, with the white car contrasting against the warm tones of the building and the blue sky. Technically, the image is well-executed, with sharp focus and good exposure. The composition is effective in creating a sense of scale and grandeur. However, the concept is somewhat straightforward, lacking a deeper narrative or symbolic layer.  While aesthetically pleasing, its originality is limited as similar images of classic cars in front of iconic buildings are common. The overall impact is one of elegant simplicity, but it may not resonate strongly with viewers seeking more conceptual depth or originality. ",
  'description': 'The image shows a white Peugeot 404 convertible parked on a cobblestone street in front of the 

In [ ]:
# prompt: move all images remaining in analysis_results from downscaled_dir to selected_dir

import shutil

# Assuming analysis_results, downscaled_dir, and selected_dir are defined as in your provided code

os.makedirs(selected_dir, exist_ok=True)

for filename, analysis in filtered_results.items():
    # if analysis.get('quality_score', 0) >= 7.5:
    source_path = os.path.join(downscaled_dir, filename)
    destination_path = os.path.join(selected_dir, filename)

    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)
        print(f"Moved {filename} to {selected_dir}")
    else:
        print(f"Warning: {filename} not found in {downscaled_dir}")

Moved P9021081.jpg to selected-pics
Moved P9021062.jpg to selected-pics


In [ ]:
#add watermark to selected images

In [ ]:
import os
from PIL import Image

def organize_images(selected_dir):
    """
    Creates a new folder for each image in the selected directory,
    moves the image into its corresponding folder, renames it to "featured",
    and creates an index.md file.
    """
    for filename in os.listdir(selected_dir):
        name, ext = os.path.splitext(filename)
        if ext.lower() in ('.jpg', '.jpeg', '.png', '.gif', '.bmp'):
          try:
            # Create a new directory for the image
            new_dir = os.path.join(selected_dir, name)
            os.makedirs(new_dir, exist_ok=True)

            # Move and rename the image
            old_path = os.path.join(selected_dir, filename)
            new_path = os.path.join(new_dir, f"featured{ext}")
            os.rename(old_path, new_path)

            print(f"Moved and renamed '{filename}' to '{new_path}'")

            # Create index.md file
            index_md_path = os.path.join(new_dir, "index.md")
            with open(index_md_path, "w") as f:
                f.write(f"# {name}\n\n")  # Example content, you can customize this
                f.write("![featured](featured{ext})\n")

            print(f"Created 'index.md' in '{new_dir}'")

          except OSError as e:
              print(f"Error processing {filename}: {e}")

# Example usage:
organize_images(selected_dir)

Moved and renamed 'P9021081.jpg' to 'selected-pics/P9021081/featured.jpg'
Created 'index.md' in 'selected-pics/P9021081'
Moved and renamed 'P9021062.jpg' to 'selected-pics/P9021062/featured.jpg'
Created 'index.md' in 'selected-pics/P9021062'


In [ ]:
from crewai import Agent, Task, Process, Crew, LLM
from crewai_tools import DirectoryReadTool, FileReadTool, FileWriterTool

llm_general = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    temperature=0.2,
    max_tokens=4096,
    api_key=userdata.get('GOOGLE_API_KEY')
    # base_url="https://api.groq.com/openai/v1"
)

In [ ]:
# @tool("MoveFiles")
# def move_file(source_path, destination_path):
#     """ Moves a file from source to destination path. """
#     try:
#         os.rename(source_path, destination_path)
#     except FileNotFoundError:
#         return "Source file not found."  # Or raise an exception
#     except Exception as e:
#         return f"Error moving file: {e}" # Or raise an exception

In [ ]:
# Define the web developer agent with tools
web_dev_agent = Agent(
    llm=llm_general,
    role="Experienced Web Developer",
    goal="Organize images and create website structure",
    backstory="""You are an experienced web developer proficient in HTML, CSS, and JavaScript.
                You are tasked with organizing image files and creating a basic website structure.
                Use the tools provided to interact with the file system.""",
    verbose=True,
    memory=True,
    tools=[DirectoryReadTool(), FileReadTool(), FileWriterTool()]  # Add the necessary tools
)

read_examples_task = Task(
    description=f"""Read all the example files from the directory '{example_web_files}'.
                    Store the content of these files in your memory for later use.""",
    expected_output="""Example files read and stored in memory.""",
    agent=web_dev_agent,
    asynchronous=True,
    input_data={"example_files_dir": example_web_files}
)

read_filtered_results_task = Task(
    description=f"""Read the content of the dictionary '{filtered_results}' and store it in your memory.
                   Remember that the keys in this dictionary correspond to the names of subfolders in '{selected_dir}'.""",
    expected_output="""'{filtered_results}' dictionary read and stored in memory.""",
    agent=web_dev_agent,
    asynchronous=True,
    input_data={
          "filtered_results": filtered_results,
          "selected_dir": selected_dir
          }
)

process_all_subfolders_task = Task(
    description=f"""Navigate to the directory '{selected_dir}'.
                   For each subfolder in this directory:
                       1. Get the name of the current subfolder. This name will match a key in the '{filtered_results}' dictionary,
                          which you should have already read and stored in memory.
                       2. Access the data in '{filtered_results}' using the subfolder name as the key.
                       3. Open the file named 'index.md' in the subfolder. If it doesn't exist, create it.
                       4. Based on the example files you read earlier, the content in '{filtered_results}', and the subfolder name,
                           write/update the 'index.md' file to have a similar structure as the example files.
                       5. Ensure the content of 'index.md' incorporates the information from '{filtered_results}' and uses a new, relevant title based on this information.
                       6. You should update the fields for title, date, and image description, based on information from '{filtered_results}'.
                       7. Repeat steps 1-6 for all subfolders in '{selected_dir}'.
                   Use DirectoryReadTool to list files, FileReadTool to read files, and FileWriterTool to update/create the 'index.md' files.
                   """,
    expected_output="""'index.md' files updated/created in all subfolders of '{selected_dir}'.""",
    agent=web_dev_agent,
    context=[read_examples_task, read_filtered_results_task],
    input_data={
        "selected_dir": selected_dir,
        "filtered_results": filtered_results,  # No longer needed here
        "example_files_dir": example_web_files
    }
)

# # Define Task 1: Organize images
# organize_images_task = Task(
#     description=f"""Navigate to the directory '{selected_dir}'.
#                    For each image in the directory, create a new subfolder named after the image (without the extension).
#                    Then, use the `move_file` tool to move the image into its corresponding subfolder and rename it to 'featured.jpg'.
#                    Use the DirectoryReadTool to list files.
#                    """,
#     expected_output="""All images organized into subfolders and renamed to 'featured.jpg'.""",
#     agent=web_dev_agent,
#     input_data={"source_path": selected_dir, "destination_path": "hello-world"},  # Pass the directory path as a string
# )


# # Define Task 2: Create index.md
# create_index_task = Task(
#     description=f"""Create a file named 'index.md' in the directory '{selected_dir}'.
#                    The content of this file should follow the structure of the example files in '{example_web_files}'.
#                    Use the image names (without extensions) as titles for each section.
#                    Use the FileReadTool to read the content of the example files.
#                    Use the tools provided to write the 'index.md' file.
#                    """,
#     expected_output="""'index.md' file created with the specified structure and content.""",
#     agent=web_dev_agent,
#     input_data={"directory": selected_dir, "example_files": example_web_files}
#     # Pass directory and example files paths as strings
# )

In [ ]:
# Define the Crew
web_dev_crew = Crew(
    agents=[web_dev_agent],
    tasks=[read_examples_task, read_filtered_results_task, process_all_subfolders_task],
    process=Process.sequential,
    output_log_file="runtime_agent_log.txt",
)


In [ ]:
# Execute the tasks
result = web_dev_crew.kickoff()
print(result)

# Agent: Experienced Web Developer
## Task: Read all the example files from the directory '/content/example-markdown'.
                    Store the content of these files in your memory for later use.


# Agent: Experienced Web Developer
## Thought: Thought:I need to list all files in the directory '/content/example-markdown' first to know which files to read.
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"/content/example-markdown\"}"
## Tool Output: 
File paths: 
-/content/example-markdown/index4.md
- /content/example-markdown/index1.md
- /content/example-markdown/index2.md
- /content/example-markdown/index5.md
- /content/example-markdown/index3.md


# Agent: Experienced Web Developer
## Thought: Thought:Now I need to read the content of each file.
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"/content/example-markdown/index4.md\"}"
## Tool Output: 
---
title: 'Big Bend National Park, April 2023 #1'
authors:
- admin
tags:
- Photog

In [ ]:
#make sure the agent is only updating the fields i want, e.g. title, date, description,
#and whether or not it's a jpg or png

In [ ]:
#example code for reference below

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai import LLM, Agent, Crew, Task, Process


llm_general = LLM(
    model="gemini/gemini-1.5-pro",
    temperature=0.5,
    max_tokens=4096,
    api_key=userdata.get('GOOGLE_API_KEY')
    # base_url="https://api.groq.com/openai/v1"
)


agent_image_extractor = Agent(
    llm = llm_general,   #getting some error message when defining llm, both for groq and gemini
    role="Image Text Extraction Specialist",
    goal="Thoroughly analyze any pictures you receive",
    backstory=(
        """You are an expert in text extraction, specializing in using AI to process and analyze textual content from images.
            Make sure you use the tools provided."""
    ),
    # tools=[search_tool, scrape_tool, pdf_search_tool, read_directory_tool],
    # tools=[VisionTool(image_path_url="https://metalbyexample.com/wp-content/uploads/figure-65.png")],
    verbose=True,
    memory=True,
)


text_extraction_task = Task(
    description="""Extract text from the provided image file. Ensure that the extracted text is accurate and complete,
    and ready for any further analysis or processing tasks. The image file provided may contain
    various text elements, so it's crucial to capture all readable text.""",
    expected_output="""A string containing the full text extracted from the image.""",
    agent=agent_image_extractor  # Assuming agent_image_extractor is defined as in your previous code
)


test_crew = Crew(
    agents=[agent_image_extractor],
    tasks=[task_hello_world],
    process=Process.sequential,
    # manager_llm = llm_bore,
    # manager_agent = manager_agent,  #optional: explicitly set a specific agent as manager instead of the manager_llm
    # cache = True,
    # memory = True,
    output_log_file="runtime-log.txt",
    # max_rpm = 2     #fix to not get rate limited by groq api
)


result = test_crew.kickoff()
print(result)